In [1]:
%pip install tappd-client web3 python-dotenv eth_account

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import hashlib

from web3 import Web3
from killswitch import Registry
from tappd_client import TappdClient
from dotenv import load_dotenv

In [3]:
load_dotenv()
tappd_client = TappdClient()

pem_key = tappd_client.derive_key('killswitchd').key
PRIVATE_KEY = hashlib.sha256(pem_key.encode()).hexdigest()
APP_ID = tappd_client.info()['app_id']
NODE_URL = 'https://eth-sepolia.g.alchemy.com/v2/' + os.getenv('ALCHEMY_API_KEY')
CONTRACT_ADDRESS = '0x684C16DD06fe0c343B461DdAD08C8537d02353A0'
CONTRACT_ABI = json.load(open('AppRegistry.json', 'r'))['abi']

# Connect to an Ethereum node
w3 = Web3(Web3.HTTPProvider(NODE_URL))
account = w3.eth.account.from_key(PRIVATE_KEY)

print(f'APP_ID: {APP_ID}')
print('='*50)
print('🔥 IMPORTANT: Gas Fee Deposit Required 🔥')
print('='*50)
print(f'Please deposit Sepolia ETH (testnet) to this address:')
print(f'\n    {account.address}\n')
print('You can get Sepolia ETH from:')
print('- https://sepoliafaucet.com')
print('='*50)

APP_ID: 6f33e6381f5fa1b217cbfb627c894cddebd3dc3f
🔥 IMPORTANT: Gas Fee Deposit Required 🔥
Please deposit Sepolia ETH (testnet) to this address:

    0xc6D246686c8e5D8AC40FA4689d1a2d7319CC54a1

You can get Sepolia ETH from:
- https://sepoliafaucet.com


In [5]:
# Initialize the registry
registry = Registry(
    wallet=account,
    contract_address=CONTRACT_ADDRESS,
    abi=CONTRACT_ABI,
    web3=w3,
    app_id=APP_ID,
    name='My App Name'
)

In [9]:
print("Registering app...")
tdx_quote = tappd_client.tdx_quote(account.address).quote
print(f"TDX quote: {tdx_quote}")
# Register app
tx_hash = registry.register(tdx_quote=tdx_quote)
print(f"Transaction hash: {tx_hash}")
# Wait for transaction receipt
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("App registered successfully!")

Registering app...
TDX quote: 0x040002008100000000000000939a7233f79c4ca9940a0db3957f06076a1f8656ef700000000000000000000000000000050102000000000000000000000000001cc6a17ab799e9a693fac7536be61c12ee1e0fabada82d0c999e08ccee2aa86de77b0870f558c570e7ffe55d6d47fa0400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000e702060000000000c68518a0ebb42136c12b2275164f8c72f25fa9a34392228687ed6e9caeb9c0f1dbd895e9cf475121c029dc47e70e91fd000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000274c2344116db7c663470693b5ba62b8621eac28cb41d2f816ddf188f9f423f900a1c44d32386fd3c993dc814e62af9d1dfbee342d1c442dbcc9c93c7d1f14129f998ab71fd4e912149daacc04c190082a94c8a7d82636393c3179f300187942000000000000000000000000

In [10]:
# Get app details
details = registry.get_app_info()
print("App info:", details)

App info: {'name': 'My App Name', 'is_banned': False, 'owner': '0xc6D246686c8e5D8AC40FA4689d1a2d7319CC54a1'}


In [11]:
print("Checking if app is banned...")
registry.check_and_shutdown()

Checking if app is banned...
App is not banned. Continuing...


False

In [12]:
print("Banning app...")
tx_hash = registry.ban_self()
print(f"Transaction hash: {tx_hash}")
# Wait for transaction receipt
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("App banned successfully!")

Banning app...
Transaction hash: be07669f8f1b07adc9702694c0079d9ffc5c24d1f4ed5305d9474e34da810085
App banned successfully!


In [13]:
print("Checking if app is banned...")
registry.check_and_shutdown()

Checking if app is banned...
App is banned. Shutting down...


True

In [15]:
print("Trying to update app name, it should fail because it is banned...")
registry.update_name('My New App Name')

Trying to update app name, it should fail because it is banned...


ContractLogicError: ('execution reverted: App is banned', '0x08c379a00000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000d4170702069732062616e6e656400000000000000000000000000000000000000')